**When will the United States reach herd immunity (>230M) for COVID-19?**

Goal: Preict the date of when the first reliable media report is published that states that >230M of the US population (~70%) have either received a SARS-CoV-2 vaccine or has been previously been infected by the virus.

Factors:
* COVID case growth rates
* Vaccines timeline (getting EUA, starting distribution, achieving sufficient scale)
* Delay between achievement and the media article

In [7]:
import math
import random

import pandas as pd
import numpy as np

from scipy import stats
from datetime import datetime, timedelta


# Model variables
GROWTH_DERIVATIVE_FACTOR_LOW = 5
GROWTH_DERIVATIVE_FACTOR_HIGH = 30

MONTHLY_GROWTH_MODIFIER_LOW = 0.5
MONTHLY_GROWTH_MODIFIER_HIGH = 4.0

DAYS_UNTIL_PEAK_GROWTH_LOW = -30
DAYS_UNTIL_PEAK_GROWTH_HIGH = 60

VACCINE_MODIFIER_LOW = 0.5
VACCINE_MODIFIER_HIGH = 1.5

RANDOM_CASE_FLUCTUATION_LOW = -2000
RANDOM_CASE_FLUCTUATION_HIGH = 4000

DAILY_FOURTH_WAVE_CHANCE = 0.6/365
FOURTH_WAVE_LENGTH_DAYS_LOW = 30
FOURTH_WAVE_LENGTH_DAYS_HIGH = 120

NEWS_ARTICLE_DELAY_DAYS_LOW = -10 # Earlier than expected based on estimated cases rather than reported cases
NEWS_ARTICLE_DELAY_DAYS_HIGH = 30

million = 1000000

# These are set to roughly curve fit the estimates from
# https://www.metaculus.com/questions/4066/10m-are-administered-an-efficacious-vaccine/
# https://www.metaculus.com/questions/5727/100m-people-vaccinated-against-covid-19/
# https://www.metaculus.com/questions/5767/1bn-people-vaccinated-against-covid-19/
MONTHLY_VACCINATION_DAILY_AMOUNTS = [np.round(5 * million * (1/30) * (1/12), 0),
                                     np.round(20 * million * (1/30) * (1/12), 0),
                                     np.round(25 * million * (1/30) * (1/12), 0),
                                     np.round(35 * million * (1/30) * (1/12), 0),
                                     np.round(40 * million * (1/30) * (1/12), 0),
                                     np.round(60 * million * (1/30) * (1/12), 0),
                                     np.round(75 * million * (1/30) * (1/12), 0),
                                     np.round(100 * million * (1/30) * (1/12), 0),
                                     np.round(150 * million * (1/30) * (1/12), 0),
                                     np.round(200 * million * (1/30) * (1/12), 0),
                                     np.round(275 * million * (1/30) * (1/12), 0),
                                     np.round(300 * million * (1/30) * (1/12), 0),
                                     np.round(330 * million * (1/30) * (1/12), 0),
                                     np.round(370 * million * (1/30) * (1/12), 0),
                                     np.round(420 * million * (1/30) * (1/12), 0),
                                     np.round(480 * million * (1/30) * (1/12), 0),
                                     np.round(560 * million * (1/30) * (1/12), 0)]

# Simulation parameters
N_SCENARIOS = 1000
SIM_START_DATE = '2020-11-28'
SIM_END_DATE = '2022-12-21'
CURRENT_COVID_CASES = 13.6 * million
CURRENT_COVID_GROWTH = 158000
CURRENT_COVID_DERIVATIVE = 2000
Q4_2020_START_COVID = 6.03 * million
TARGET_POP = 230 * million
US_POP = 328.2 * million

In [8]:
def print_pct(pct, digits=0):
    pct = pct * 100
    pct = np.round(pct, digits)
    if pct >= 100:
        if digits == 0:
            val = '>99.0%'
        else:
            val = '>99.'
            for d in range(digits - 1):
                val += '9'
            val += '9%'
    elif pct <= 0:
        if digits == 0:
            val = '<1.0%'
        else:
            val = '<0.'
            for d in range(digits - 1):
                val += '0'
            val += '1%'
    else:
        val = '{}%'.format(pct)
    return val


def get_metaculus_range(rangex, minx=None, maxx=None):
    lower25, mean, upper75 = np.percentile(rangex, [25, 50, 75])
    results = {}
    results['lower25'] = lower25
    results['mean'] = mean
    results['upper75'] = upper75
    if minx:
        results['minx'] = print_pct(sum([x < minx for x in rangex]) / len(rangex))
    if maxx:
        results['maxx'] = print_pct(sum([x > maxx for x in rangex]) / len(rangex))
    return results
    

def normal_sample(low, high, interval):
    if (low > high) or (high < low):
        raise ValueError
    if low == high:
        return low
    else:
        mu = (high + low) / 2
        cdf_value = 0.5 + 0.5 * interval
        normed_sigma = stats.norm.ppf(cdf_value)
        sigma = (high - mu) / normed_sigma
        return np.random.normal(mu, sigma)

    
def lognormal_sample(low, high, interval):
    if (low > high) or (high < low):
        raise ValueError
    if low == high:
        return low
    else:
        log_low = np.log(low)
        log_high = np.log(high)
        mu = (log_high + log_low) / 2
        cdf_value = 0.5 + 0.5 * interval
        normed_sigma = stats.norm.ppf(cdf_value)
        sigma = (log_high - mu) / normed_sigma
        return np.random.lognormal(mu, sigma)

In [9]:
days_until_sim_end_date = (datetime.strptime(SIM_END_DATE, '%Y-%m-%d') - datetime.strptime(SIM_START_DATE, '%Y-%m-%d')).days + 1

dec_7_2020_covid_collector = []
y2020_covid_collector = []
q1_2021_covid_collector = []
q2_2021_covid_collector = []
q3_2021_covid_collector = []
q4_2021_covid_collector = []
q1_2021_vaccinated_collector = []
q2_2021_vaccinated_collector = []
q3_2021_vaccinated_collector = []
q4_2021_vaccinated_collector = []
q1_2021_target_collector = []
q2_2021_target_collector = []
q3_2021_target_collector = []
q4_2021_target_collector = []
target_date_collector = []


for n in range(N_SCENARIOS):
    done = False
    fourth_wave = False
    today = datetime.today()
    
    covid = CURRENT_COVID_CASES
    covid_growth = CURRENT_COVID_GROWTH
    
    growth_derivative_factor = np.round(normal_sample(GROWTH_DERIVATIVE_FACTOR_LOW, GROWTH_DERIVATIVE_FACTOR_HIGH, 0.8) + 1, 1)
    if growth_derivative_factor < 0.3:
        growth_derivative_factor = 0.3
    
    monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
    if monthly_growth_modifier < 0.5:
        monthly_growth_modifier = 0.5
     
    peak_growth_days = np.round(normal_sample(DAYS_UNTIL_PEAK_GROWTH_LOW, DAYS_UNTIL_PEAK_GROWTH_HIGH, 0.8), 0)
    if peak_growth_days < 0:
        peak_growth_days = 0
    peak_growth_date = (today + timedelta(days=peak_growth_days)).date()
    
    fourth_wave_days_length = np.round(lognormal_sample(FOURTH_WAVE_LENGTH_DAYS_LOW, FOURTH_WAVE_LENGTH_DAYS_HIGH, 0.8), 0)
    if fourth_wave_days_length < 1:
        fourth_wave_days_length = 1
        
    vaccine_modifier = np.round(normal_sample(VACCINE_MODIFIER_LOW, VACCINE_MODIFIER_HIGH, 0.8), 1)
    if vaccine_modifier < 0.3:
        vaccine_modifier = 0.3
        
    news_article_delay_days = np.round(normal_sample(NEWS_ARTICLE_DELAY_DAYS_LOW, NEWS_ARTICLE_DELAY_DAYS_HIGH, 0.8), 1)
    
    vaccine_rate = 0
    total_vaccinated = 0
    
    verbose = (n <= 4) or (n % 10 == 0)
    really_verbose = (n <= 4)
    
    if verbose:
        print('-')
        print('-')
        print('## SCENARIO {}/{} ##'.format(n + 1, N_SCENARIOS))
        print('Growth derivative factor: {}'.format(growth_derivative_factor))
        print('Peak growth on: {}'.format(peak_growth_date))
        print('News article delay days: {}'.format(news_article_delay_days))

    for day in range(days_until_sim_end_date):
        date = str((today + timedelta(days = day)).date())
        if really_verbose:
            print('Day {}: {}'.format(day, date))
            
        # Grow vaccines
        days_since_vaccine_start = (today + timedelta(days = day) - datetime(2021, 1, 1)).days
        if days_since_vaccine_start >= 0:
            i = math.floor(days_since_vaccine_start / 30)
            if i >= len(MONTHLY_VACCINATION_DAILY_AMOUNTS):
                i = len(MONTHLY_VACCINATION_DAILY_AMOUNTS) - 1
            daily_vaccines = MONTHLY_VACCINATION_DAILY_AMOUNTS[i]
            old_vaccine_rate = vaccine_rate
            vaccine_rate = np.round(np.random.normal(daily_vaccines  * vaccine_modifier,
                                                     daily_vaccines  * vaccine_modifier), 0)
            if vaccine_rate < old_vaccine_rate:
                vaccine_rate = old_vaccine_rate
            total_vaccinated = np.round(total_vaccinated + vaccine_rate, 0)
            
        if really_verbose:
            print('...Total vaccinated now {} (+{})'.format(total_vaccinated, vaccine_rate))
            
        if really_verbose:
            print('...Total vaccinated now {} (+{})'.format(total_vaccinated, vaccine_rate))
            
            
        # Grow COVID
        if day > peak_growth_days and (not fourth_wave or day > fourth_wave_end):
            covid_growth -= CURRENT_COVID_DERIVATIVE / growth_derivative_factor * (1 / monthly_growth_modifier)
        else:
            covid_growth += (CURRENT_COVID_DERIVATIVE / growth_derivative_factor) * monthly_growth_modifier
        covid_growth = np.round(covid_growth, 0)
        old_covid_growth = covid_growth
        if old_covid_growth < 0:
            old_covid_growth = 0
            
        covid_growth -= np.round(vaccine_rate * 0.02, 0)
        if covid_growth < 0:
            covid_growth = 0
            
        random_case_fluctuation = np.round(normal_sample(RANDOM_CASE_FLUCTUATION_LOW, RANDOM_CASE_FLUCTUATION_HIGH, 0.8), 0)
        covid += covid_growth + random_case_fluctuation
        current_pop = covid + total_vaccinated
        current_target = current_pop / US_POP * 100
        if current_pop >= TARGET_POP and not done:
            target_date_collector.append(day + news_article_delay_days)
            done = True
            
        if really_verbose:
            print('...Total COVID cases now {} (+{} {} -{} *{})'.format(covid,
                                                                        old_covid_growth,
                                                                        "%+d" % random_case_fluctuation,
                                                                        np.round(vaccine_rate * 0.02, 0),
                                                                        monthly_growth_modifier))
            print('...% of target: {}%/{}%'.format(np.round(current_target, 1),
                                                   np.round(TARGET_POP / US_POP * 100, 1)))
                
        if not fourth_wave and np.random.random() <= DAILY_FOURTH_WAVE_CHANCE:
            if really_verbose:
                print('...COVID FOURTH WAVE!!!')
            fourth_wave = True
            fourth_wave_end = day + fourth_wave_days_length
            
            
        # Record
        if date == '2020-12-07':
            dec7_covid = covid
            if verbose:
                print('...COVID by 7 Dec: {}'.format(dec7_covid))
                    
        if date == '2020-12-31':
            y2020_covid = covid
            if verbose:
                print('...Total 2020 COVID: {}'.format(covid))
                
        if date.split('-')[-1] == '01':
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
                
        if date == '2021-03-31':
            q1_2021_covid = covid
            q1_2021_vaccinated = total_vaccinated
            q1_2021_target = np.round((covid + total_vaccinated) / US_POP * 100, 1)
            if verbose:
                print('...Q1 2021 COVID: {}'.format(q1_2021_covid))
                print('...Q1 2021 Vaccinated: {}'.format(q1_2021_vaccinated))
                print('...Q1 2021 Target: {}'.format(q1_2021_target))

        if date == '2021-06-30':
            q2_2021_covid = covid
            q2_2021_vaccinated = total_vaccinated
            q2_2021_target = np.round((covid + total_vaccinated) / US_POP * 100, 1)
            if verbose:
                print('...Q2 2021 COVID: {}'.format(q2_2021_covid))
                print('...Q2 2021 Vaccinated: {}'.format(q2_2021_vaccinated))
                print('...Q2 2021 Target: {}'.format(q2_2021_target))
            
        if date == '2021-09-30':
            q3_2021_covid = covid
            q3_2021_vaccinated = total_vaccinated
            q3_2021_target = np.round((covid + total_vaccinated) / US_POP * 100, 1)
            if verbose:
                print('...Q3 2021 COVID: {}'.format(q3_2021_covid))
                print('...Q3 2021 Vaccinated: {}'.format(q3_2021_vaccinated))
                print('...Q3 2021 Target: {}'.format(q3_2021_target))
                
        if date == '2021-12-31':
            q4_2021_covid = covid
            q4_2021_vaccinated = total_vaccinated
            q4_2021_target = np.round((covid + total_vaccinated) / US_POP * 100, 1)
            if verbose:
                print('...Q4 2021 COVID: {}'.format(q4_2021_covid))
                print('...Q4 2021 Vaccinated: {}'.format(q4_2021_vaccinated))
                print('...Q4 2021 Target: {}'.format(q4_2021_target))
   
    if not done:
        target_date_collector.append(1000)

    dec_7_2020_covid_collector.append(dec7_covid)
    y2020_covid_collector.append(y2020_covid)
    q1_2021_covid_collector.append(q1_2021_covid)
    q2_2021_covid_collector.append(q2_2021_covid)
    q3_2021_covid_collector.append(q3_2021_covid)
    q4_2021_covid_collector.append(q4_2021_covid)
    q1_2021_vaccinated_collector.append(q1_2021_vaccinated)
    q2_2021_vaccinated_collector.append(q2_2021_vaccinated)
    q3_2021_vaccinated_collector.append(q3_2021_vaccinated)
    q4_2021_vaccinated_collector.append(q4_2021_vaccinated)
    q1_2021_target_collector.append(q1_2021_target)
    q2_2021_target_collector.append(q2_2021_target)
    q3_2021_target_collector.append(q3_2021_target)
    q4_2021_target_collector.append(q4_2021_target)

-
-
## SCENARIO 1/1000 ##
Growth derivative factor: 16.2
Peak growth on: 2020-12-02
News article delay days: 15.8
Day 0: 2020-12-02
...Total vaccinated now 0 (+0)
...Total vaccinated now 0 (+0)
...Total COVID cases now 13757279.0 (+158802.0 -1523 -0.0 *6.5)
...% of target: 4.2%/70.1%
Day 1: 2020-12-03
...Total vaccinated now 0 (+0)
...Total vaccinated now 0 (+0)
...Total COVID cases now 13916215.0 (+158783.0 +153 -0.0 *6.5)
...% of target: 4.2%/70.1%
Day 2: 2020-12-04
...Total vaccinated now 0 (+0)
...Total vaccinated now 0 (+0)
...Total COVID cases now 14076554.0 (+158764.0 +1575 -0.0 *6.5)
...% of target: 4.3%/70.1%
Day 3: 2020-12-05
...Total vaccinated now 0 (+0)
...Total vaccinated now 0 (+0)
...Total COVID cases now 14236797.0 (+158745.0 +1498 -0.0 *6.5)
...% of target: 4.3%/70.1%
Day 4: 2020-12-06
...Total vaccinated now 0 (+0)
...Total vaccinated now 0 (+0)
...Total COVID cases now 14398094.0 (+158726.0 +2571 -0.0 *6.5)
...% of target: 4.4%/70.1%
Day 5: 2020-12-07
...Total vacci

...Total COVID cases now 24090294.0 (+0 -176 -29047.0 *1.9)
...% of target: 54.4%/70.1%
Day 277: 2021-09-05
...Total vaccinated now 155749952.0 (+1452347.0)
...Total vaccinated now 155749952.0 (+1452347.0)
...Total COVID cases now 24092888.0 (+0 +2594 -29047.0 *1.9)
...% of target: 54.8%/70.1%
Day 278: 2021-09-06
...Total vaccinated now 157202299.0 (+1452347.0)
...Total vaccinated now 157202299.0 (+1452347.0)
...Total COVID cases now 24090931.0 (+0 -1957 -29047.0 *1.9)
...% of target: 55.2%/70.1%
Day 279: 2021-09-07
...Total vaccinated now 158654646.0 (+1452347.0)
...Total vaccinated now 158654646.0 (+1452347.0)
...Total COVID cases now 24092844.0 (+0 +1913 -29047.0 *1.9)
...% of target: 55.7%/70.1%
Day 280: 2021-09-08
...Total vaccinated now 160688725.0 (+2034079.0)
...Total vaccinated now 160688725.0 (+2034079.0)
...Total COVID cases now 24095292.0 (+0 +2448 -40682.0 *1.9)
...% of target: 56.3%/70.1%
Day 281: 2021-09-09
...Total vaccinated now 163513923.0 (+2825198.0)
...Total vaccin

...Total vaccinated now 1027636949.0 (+6189510.0)
...Total vaccinated now 1027636949.0 (+6189510.0)
...Total COVID cases now 24257475.0 (+0 -6234 -123790.0 *1.9)
...% of target: 320.5%/70.1%
Day 476: 2022-03-23
...Total vaccinated now 1033826459.0 (+6189510.0)
...Total vaccinated now 1033826459.0 (+6189510.0)
...Total COVID cases now 24257187.0 (+0 -288 -123790.0 *1.9)
...% of target: 322.4%/70.1%
Day 477: 2022-03-24
...Total vaccinated now 1040015969.0 (+6189510.0)
...Total vaccinated now 1040015969.0 (+6189510.0)
...Total COVID cases now 24257859.0 (+0 +672 -123790.0 *1.9)
...% of target: 324.3%/70.1%
Day 478: 2022-03-25
...Total vaccinated now 1046205479.0 (+6189510.0)
...Total vaccinated now 1046205479.0 (+6189510.0)
...Total COVID cases now 24253889.0 (+0 -3970 -123790.0 *1.9)
...% of target: 326.2%/70.1%
Day 479: 2022-03-26
...Total vaccinated now 1052394989.0 (+6189510.0)
...Total vaccinated now 1052394989.0 (+6189510.0)
...Total COVID cases now 24257683.0 (+0 +3794 -123790.0 *1

...Total vaccinated now 2609397011.0 (+9537610.0)
...Total vaccinated now 2609397011.0 (+9537610.0)
...Total COVID cases now 24478615.0 (+0 +2963 -190752.0 *2.7)
...% of target: 802.5%/70.1%
Day 676: 2022-10-09
...Total vaccinated now 2618934621.0 (+9537610.0)
...Total vaccinated now 2618934621.0 (+9537610.0)
...Total COVID cases now 24477184.0 (+0 -1431 -190752.0 *2.7)
...% of target: 805.4%/70.1%
Day 677: 2022-10-10
...Total vaccinated now 2628472231.0 (+9537610.0)
...Total vaccinated now 2628472231.0 (+9537610.0)
...Total COVID cases now 24477499.0 (+0 +315 -190752.0 *2.7)
...% of target: 808.3%/70.1%
Day 678: 2022-10-11
...Total vaccinated now 2638009841.0 (+9537610.0)
...Total vaccinated now 2638009841.0 (+9537610.0)
...Total COVID cases now 24478394.0 (+0 +895 -190752.0 *2.7)
...% of target: 811.2%/70.1%
Day 679: 2022-10-12
...Total vaccinated now 2647547451.0 (+9537610.0)
...Total vaccinated now 2647547451.0 (+9537610.0)
...Total COVID cases now 24481135.0 (+0 +2741 -190752.0 *2

...Total vaccinated now 4090222.0 (+99790.0)
...Total COVID cases now 29753939.0 (+76056.0 +2971 -1996.0 *3.2)
...% of target: 10.3%/70.1%
...Q1 2021 COVID: 29753939.0
...Q1 2021 Vaccinated: 4090222.0
...Q1 2021 Target: 10.3
Day 120: 2021-04-01
...Total vaccinated now 4190012.0 (+99790.0)
...Total vaccinated now 4190012.0 (+99790.0)
...Total COVID cases now 29824736.0 (+74036.0 -1243 -1996.0 *3.2)
...% of target: 10.4%/70.1%
Day 121: 2021-04-02
...Total vaccinated now 4289802.0 (+99790.0)
...Total vaccinated now 4289802.0 (+99790.0)
...Total COVID cases now 29900082.0 (+72003.0 +5339 -1996.0 *2.1)
...% of target: 10.4%/70.1%
Day 122: 2021-04-03
...Total vaccinated now 4389592.0 (+99790.0)
...Total vaccinated now 4389592.0 (+99790.0)
...Total COVID cases now 29970503.0 (+69970.0 +2447 -1996.0 *2.1)
...% of target: 10.5%/70.1%
Day 123: 2021-04-04
...Total vaccinated now 4511989.0 (+122397.0)
...Total vaccinated now 4511989.0 (+122397.0)
...Total COVID cases now 30037242.0 (+67937.0 +1250

...

-
-
## SCENARIO 331/1000 ##
Growth derivative factor: 13.2
Peak growth on: 2021-01-20
News article delay days: 18.6
...COVID by 7 Dec: 14558793.0
...Total 2020 COVID: 18531715.0
...Q1 2021 COVID: 28750697.0
...Q1 2021 Vaccinated: 8088747.0
...Q1 2021 Target: 11.2
...Q2 2021 COVID: 28816077.0
...Q2 2021 Vaccinated: 30770172.0
...Q2 2021 Target: 18.2
...Q3 2021 COVID: 28950882.0
...Q3 2021 Vaccinated: 95052303.0
...Q3 2021 Target: 37.8
...Q4 2021 COVID: 29017214.0
...Q4 2021 Vaccinated: 216371947.0
...Q4 2021 Target: 74.8
-
-
## SCENARIO 341/1000 ##
Growth derivative factor: 2.9
Peak growth on: 2020-12-24
News article delay days: -19.0
...COVID by 7 Dec: 14600647.0
...Total 2020 COVID: 19259834.0
...Q1 2021 COVID: 30178712.0
...Q1 2021 Vaccinated: 9536250.0
...Q1 2021 Target: 12.1
...Q2 2021 COVID: 30243260.0
...Q2 2021 Vaccinated: 39819480.0
...Q2 2021 Target: 21.3
...Q3 2021 COVID: 30337789.0
...Q3 2021 Vaccinated: 100908632.0
...Q3 2021 Target: 40.0
...Q4 2021 COVID: 30470410.0
...Q4 

-
-
## SCENARIO 491/1000 ##
Growth derivative factor: 26.8
Peak growth on: 2020-12-02
News article delay days: -3.5
...COVID by 7 Dec: 14552512.0
...Total 2020 COVID: 18451612.0
...Q1 2021 COVID: 32235230.0
...Q1 2021 Vaccinated: 3255427.0
...Q1 2021 Target: 10.8
...Q2 2021 COVID: 37359577.0
...Q2 2021 Vaccinated: 13819721.0
...Q2 2021 Target: 15.6
...Q3 2021 COVID: 37496277.0
...Q3 2021 Vaccinated: 39044666.0
...Q3 2021 Target: 23.3
...Q4 2021 COVID: 37590148.0
...Q4 2021 Vaccinated: 97785160.0
...Q4 2021 Target: 41.2
-
-
## SCENARIO 501/1000 ##
Growth derivative factor: 13.2
Peak growth on: 2021-01-26
News article delay days: 26.9
...COVID by 7 Dec: 14543982.0
...Total 2020 COVID: 18392044.0
...Q1 2021 COVID: 25992710.0
...Q1 2021 Vaccinated: 12548967.0
...Q1 2021 Target: 11.7
...Q2 2021 COVID: 26101880.0
...Q2 2021 Vaccinated: 55288402.0
...Q2 2021 Target: 24.8
...Q3 2021 COVID: 26153053.0
...Q3 2021 Vaccinated: 140630025.0
...Q3 2021 Target: 50.8
...Q4 2021 COVID: 26295896.0
...Q4 

-
-
## SCENARIO 651/1000 ##
Growth derivative factor: 8.6
Peak growth on: 2020-12-19
News article delay days: 21.9
...COVID by 7 Dec: 14558042.0
...Total 2020 COVID: 18541306.0
...Q1 2021 COVID: 27321834.0
...Q1 2021 Vaccinated: 9659613.0
...Q1 2021 Target: 11.3
...Q2 2021 COVID: 27401752.0
...Q2 2021 Vaccinated: 34990187.0
...Q2 2021 Target: 19.0
...Q3 2021 COVID: 27495807.0
...Q3 2021 Vaccinated: 91508539.0
...Q3 2021 Target: 36.3
...Q4 2021 COVID: 27616041.0
...Q4 2021 Vaccinated: 253932637.0
...Q4 2021 Target: 85.8
-
-
## SCENARIO 661/1000 ##
Growth derivative factor: 5.7
Peak growth on: 2020-12-18
News article delay days: 0.9
...COVID by 7 Dec: 14575766.0
...Total 2020 COVID: 18821681.0
...Q1 2021 COVID: 26569923.0
...Q1 2021 Vaccinated: 13836672.0
...Q1 2021 Target: 12.3
...Q2 2021 COVID: 26656206.0
...Q2 2021 Vaccinated: 64340913.0
...Q2 2021 Target: 27.7
...Q3 2021 COVID: 26762439.0
...Q3 2021 Vaccinated: 193329442.0
...Q3 2021 Target: 67.1
...Q4 2021 COVID: 26861751.0
...Q4 20

-
-
## SCENARIO 811/1000 ##
Growth derivative factor: 16.7
Peak growth on: 2020-12-02
News article delay days: 7.8
...COVID by 7 Dec: 14557819.0
...Total 2020 COVID: 18379809.0
...Q1 2021 COVID: 26502691.0
...Q1 2021 Vaccinated: 10512861.0
...Q1 2021 Target: 11.3
...Q2 2021 COVID: 26574215.0
...Q2 2021 Vaccinated: 36026246.0
...Q2 2021 Target: 19.1
...Q3 2021 COVID: 26670294.0
...Q3 2021 Vaccinated: 103101436.0
...Q3 2021 Target: 39.5
...Q4 2021 COVID: 26791368.0
...Q4 2021 Vaccinated: 270677094.0
...Q4 2021 Target: 90.6
-
-
## SCENARIO 821/1000 ##
Growth derivative factor: 8.8
Peak growth on: 2020-12-02
News article delay days: 23.9
...COVID by 7 Dec: 14559196.0
...Total 2020 COVID: 18369597.0
...Q1 2021 COVID: 25453217.0
...Q1 2021 Vaccinated: 12582778.0
...Q1 2021 Target: 11.6
...Q2 2021 COVID: 25552021.0
...Q2 2021 Vaccinated: 45024720.0
...Q2 2021 Target: 21.5
...Q3 2021 COVID: 25623560.0
...Q3 2021 Vaccinated: 125840559.0
...Q3 2021 Target: 46.1
...Q4 2021 COVID: 25678608.0
...Q4

-
-
## SCENARIO 971/1000 ##
Growth derivative factor: 20.2
Peak growth on: 2020-12-20
News article delay days: 48.5
...COVID by 7 Dec: 14568118.0
...Total 2020 COVID: 18568802.0
...Q1 2021 COVID: 27705255.0
...Q1 2021 Vaccinated: 9312118.0
...Q1 2021 Target: 11.3
...Q2 2021 COVID: 27739833.0
...Q2 2021 Vaccinated: 34370328.0
...Q2 2021 Target: 18.9
...Q3 2021 COVID: 27833470.0
...Q3 2021 Vaccinated: 92616690.0
...Q3 2021 Target: 36.7
...Q4 2021 COVID: 27907474.0
...Q4 2021 Vaccinated: 246377639.0
...Q4 2021 Target: 83.6
-
-
## SCENARIO 981/1000 ##
Growth derivative factor: 8.1
Peak growth on: 2021-02-06
News article delay days: 17.6
...COVID by 7 Dec: 14556793.0
...Total 2020 COVID: 18692508.0
...Q1 2021 COVID: 28540622.0
...Q1 2021 Vaccinated: 16063230.0
...Q1 2021 Target: 13.6
...Q2 2021 COVID: 28608883.0
...Q2 2021 Vaccinated: 60683229.0
...Q2 2021 Target: 27.2
...Q3 2021 COVID: 28712609.0
...Q3 2021 Vaccinated: 178355930.0
...Q3 2021 Target: 63.1
...Q4 2021 COVID: 28811474.0
...Q4 

## Total US COVID by 7 Dec

For calibration

In [10]:
get_metaculus_range(dec_7_2020_covid_collector)

{'lower25': 14555283.0, 'mean': 14560312.5, 'upper75': 14566743.75}

## By EOY 2020

In [11]:
get_metaculus_range(y2020_covid_collector)

{'lower25': 18374535.5, 'mean': 18458127.5, 'upper75': 18560888.5}

## By EOQ Q1 2021

In [12]:
get_metaculus_range(q1_2021_covid_collector)

{'lower25': 25718988.25, 'mean': 26967654.5, 'upper75': 28762972.75}

In [13]:
get_metaculus_range(q1_2021_vaccinated_collector)

{'lower25': 8208965.0, 'mean': 11161710.5, 'upper75': 14130882.5}

In [14]:
get_metaculus_range(q1_2021_target_collector)

{'lower25': 11.1, 'mean': 11.6, 'upper75': 12.4}

## By EOQ Q2 2021

In [15]:
get_metaculus_range(q2_2021_covid_collector)

{'lower25': 25820573.75, 'mean': 27080088.0, 'upper75': 28911101.25}

In [16]:
get_metaculus_range(q2_2021_vaccinated_collector)

{'lower25': 30691303.75, 'mean': 42539985.5, 'upper75': 54618412.5}

In [17]:
get_metaculus_range(q2_2021_target_collector)

{'lower25': 18.3, 'mean': 21.35, 'upper75': 25.1}

## By EOQ Q3 2021

In [18]:
get_metaculus_range(q3_2021_covid_collector)

{'lower25': 25917888.75, 'mean': 27166817.5, 'upper75': 29013381.5}

In [19]:
get_metaculus_range(q3_2021_vaccinated_collector)

{'lower25': 89553577.5, 'mean': 120508820.0, 'upper75': 152778862.5}

In [20]:
get_metaculus_range(q3_2021_target_collector)

{'lower25': 36.3, 'mean': 45.45, 'upper75': 55.449999999999996}

## By EOY 2021

In [21]:
get_metaculus_range(q4_2021_covid_collector)

{'lower25': 26002368.5, 'mean': 27269530.0, 'upper75': 29088585.75}

In [22]:
get_metaculus_range(q4_2021_vaccinated_collector)

{'lower25': 228176703.25, 'mean': 312590052.5, 'upper75': 397261566.5}

In [23]:
get_metaculus_range(q4_2021_target_collector)

{'lower25': 78.77499999999999, 'mean': 104.1, 'upper75': 130.7}

## Target Date

In [24]:
'target not exceeded by 2022-12-21 in {} of scenarios'.format(print_pct(sum([x == days_until_sim_end_date for x in target_date_collector]) / len(target_date_collector)))

'target not exceeded by 2022-12-21 in <1.0% of scenarios'

In [25]:
results = get_metaculus_range([x for x in target_date_collector if x < days_until_sim_end_date])
results['lower25'] = str((datetime.strptime(SIM_START_DATE, '%Y-%m-%d') + timedelta(days=results['lower25'])).date())
results['mean'] = str((datetime.strptime(SIM_START_DATE, '%Y-%m-%d') + timedelta(days=results['mean'])).date())
results['upper75'] = str((datetime.strptime(SIM_START_DATE, '%Y-%m-%d') + timedelta(days=results['upper75'])).date())
results

{'lower25': '2021-10-29', 'mean': '2021-11-24', 'upper75': '2021-12-24'}